In [16]:
import math
import tensorflow as tf

In [28]:
def create_layer(scope_name, input, layer_size, activation=None, trainable=True):
  with tf.variable_scope(scope_name):
    input_size = input.get_shape()[1]
    w = tf.get_variable("weights", (input_size, layer_size), trainable=trainable, dtype=tf.float32)
    b = tf.get_variable("bias", (layer_size,), initializer=tf.zeros_initializer(),
                        trainable=trainable, dtype=tf.float32)
    output = tf.matmul(input, w) + b
    if activation:
      output = activation(output)
    return output, w, b

def create_dnn(name, input, layer_sizes, actions, trainable=True):
  with tf.variable_scope(name):
    previous = input
    variables = []
    for i, layer_size in enumerate(layer_sizes):
      previous, w, b = create_layer("Hidden_{}".format(i), previous,
                                    layer_size, activation=tf.nn.relu,
                                    trainable=trainable)
      variables += [w, b]
    
    logits, w, b = create_layer("Logits", previous,
                                actions, activation=None,
                                trainable=trainable)
    variables += [w, b]
    action_distribution = tf.nn.softmax(logits)
  return action_distribution, variables


In [29]:
g = tf.Graph()
with g.as_default():
  input = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=tf.float32)
  a, v = create_dnn("dqn", input, [10, 5], 3)  

In [30]:
tf.summary.FileWriter("logs", g).close()